### 실습 개요
- 이미지에서 얼굴을 찾습니다.
- 얼굴 이미지들간의 유사한 정도를 측정합니다.
- 이미지에서 특정인의 얼굴을 찾아 다른 사람의 얼굴로 바꿉니다.

### 사전준비
#### 1. 필수 Library들을 설치합니다.
- **OpenCV**: 이미지와 비디오 분석을 위한 컴퓨터 비전 라이브러리. 얼굴 검출, 객체 추적, 이미지 처리에 널리 사용됨.

- **InsightFace**: 얼굴 인식 및 검출을 위한 오픈 소스 라이브러리. ArcFace 기반의 높은 정확도로 보안 및 인증 시스템에 활용됨.

- **ONNX Runtime**: ONNX 형식의 딥러닝 모델을 빠르고 효율적으로 실행하는 런타임.

In [1]:
!pip install insightface onnxruntime


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


#### 2. AI 모델 설치
##### 2-1. Facial Anaysis 모델 설치
- buffalo_l download from : 
  
    https://github.com/deepinsight/insightface/releases

- `C:\models\buffalo_l`에 `buffalo_l.zip` 을 unzip 하고 전역변수에 아래와 같이 저장 위치 명시


In [2]:
BUFFALO_L_PATH = "C:\\"

##### 2-2. Face Swap 모델 설치
- Download `inswapper_128.onnx` :
  
    https://huggingface.co/ezioruan/inswapper_128.onnx/tree/main

- 전역변수에 저장 위치 명시

In [3]:
INSWAPPER_PATH = r"C:\models\inswapper_128.onnx"

##### 2-3. 모델 설치 결과
```bash
C:\models
├── inswapper_128.onnx
└── buffalo_l
    ├── 1k3d68.onnx
    ├── 2d106det.onnx
    ├── det_10g.onnx
    ├── genderage.onnx
    └── w600k_r50.onnx
```

### 공부하기

#### 1. Yaw, Pitch, Roll
**Yaw, Pitch, Roll**는 얼굴의 3D 방향을 나타내는 각도 값입니다:

1. **Yaw (요)**: 얼굴이 좌우로 회전할 때의 각도를 나타냅니다.
2. **Pitch (피치)**: 얼굴이 위나 아래를 볼 때의 각도를 나타냅니다.
3. **Roll (롤)**: 머리가 한쪽으로 기울어졌을 때의 각도를 나타냅니다.

![Link](./faces/yaw_pitch_roll.jpg)

#### 2. 중요한 Class 들 불러오기
- **`FaceAnalysis`**: 얼굴 분석을 위한 클래스입니다. 얼굴 감지 모델을 초기화하고, 입력 이미지에서 얼굴을 찾고 다양한 속성(성별, 나이, 얼굴 특징 등)을 추출하는 데 사용됩니다.

- **`Face`**: 얼굴의 세부 정보를 담고 있는 객체입니다. 감지된 얼굴의 경계 상자(`bbox`), 랜드마크(`landmark`), 특징 벡터(`embedding`), 성별, 나이 등 얼굴에 관한 다양한 속성을 포함합니다.

In [4]:
from insightface.app import FaceAnalysis
from insightface.app.common import Face

c:\pypjt\face2\Lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


### 얼굴 분석하기: 이미지에서 얼굴을 찾고, 분석하고, 시각화하기
#### 1. Class `Facama` 정의
- 생성자 `__init__`: Face Analysis 모델을 메모리에 로드하고 AI 기능을 초기화합니다.
- 메서드 `get_faces`: 입력 이미지에서 얼굴을 감지하고, 특징을 추출한 얼굴 정보(`Face` 객체들의 `list`)를 반환합니다.
- 메서드 `detect_and_process_faces`: 이미지에서 얼굴을 감지하고, 각 얼굴에 사용자 정의 처리를 적용한 후 수정된 이미지를 반환합니다.

In [5]:
import numpy as np
from typing import Callable

class Facama:

    # Face Analysis 모델을 메모리에 로드하고 AI 기능을 초기화
    def __init__(
            self, 
            name='buffalo_l', # 사용할 얼굴 인식 모델 이름.
            root=BUFFALO_L_PATH, 
            ctx_id=-1, # 컨텍스트 ID. -1은 CPU, 0 이상은 GPU ID.
            nms_thresh = 0.6,  # NMS 임계값.
        ):

        # FaceAnalysis 객체 초기화
        self.app = FaceAnalysis(name=name, root=root) 
        self.app.prepare(ctx_id=ctx_id)

        # NMS 임계값 설정
        self.app.det_model.nms_thresh = nms_thresh

    def get_faces(self, input_image: np.ndarray) -> list[Face]:

        # 얼굴 임베딩 추출
        faces = self.app.get(input_image)
        if not faces:
            raise ValueError("기준 이미지에서 얼굴을 검출하지 못했습니다.")
    
        # 얼굴(들) 반환
        return faces

    def detect_and_process_faces(
            self,
            input_image: np.ndarray,
            f: Callable[[np.ndarray, Face], None],
        ) -> np.ndarray:

        faces = self.get_faces(input_image)

        # 검출된 얼굴 처리
        for face in faces:
            f(input_image, face)
        
        # 처리된 이미지를 반환
        return input_image

#### 2. Class `Facama` 인스턴스 생성

In [6]:
# Facama 인스턴스 생성
facama = Facama()

c:\pypjt\face2\Lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


#### 3. 얼굴에 Box 그리기
- 얼굴 영역에 Box 그리는 콜백 함수(callback function) `draw_face_bbox` 정의
- 함수 `detect_and_process_faces`에 원본 이미지, `draw_face_bbox`를 전달하여 실행

In [7]:
import cv2

# 얼굴 영역에 Box 그리기 함수 정의
def draw_face_bbox(input_image: np.ndarray, face: Face) -> None:
    # 얼굴 영역 표시
    bbox = face.bbox.astype(int)
    cv2.rectangle(input_image, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 2)

# 이미지 파일 읽기
img = cv2.imread("./faces/newJeans.jpg")
if img is None:
    raise FileNotFoundError("이미지를 불러올 수 없습니다. 경로를 확인하세요.")
    
# 얼굴 검출 및 처리
result_img = facama.detect_and_process_faces(
    img,
    draw_face_bbox,
)
    
# 결과 이미지 저장
output_path = 'result_bbox.jpg'  # 저장할 경로와 파일 이름
cv2.imwrite(output_path, result_img)

NameError: name 'cv2' is not defined

#### 4. 얼굴 별로 Yaw, Pitch, Roll 값 출력하기
- Yaw, Pitch, Roll 값 출력하는 콜백 함수(callback function) `draw_ypr` 정의
- 함수 `detect_and_process_faces`에 원본 이미지, `draw_ypr`를 전달하여 실행

In [ ]:
# 얼굴 별로 Yaw, Pitch, Roll 출력하기 함수 정의
def draw_ypr(input_image: np.ndarray, face: Face) -> None:
    # 얼굴 영역 표시
    bbox = face.bbox.astype(int)
    cv2.rectangle(img, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 2)
    
    # YAW, PITCH, ROLL 계산
    yaw, pitch, roll = face.pose[1], face.pose[0], face.pose[2]

    # YAW, PITCH, ROLL 값 표시
    if yaw is not None and pitch is not None and roll is not None:
        cv2.putText(img, f"YAW: {yaw:.1f}", (bbox[0] + 5, bbox[1] + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (127, 127, 127), 2)
        cv2.putText(img, f"PITCH: {pitch:.1f}", (bbox[0] + 5, bbox[1] + 45), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (127, 127, 127), 2)
        cv2.putText(img, f"ROLL: {roll:.1f}", (bbox[0] + 5, bbox[1] + 70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (127, 127, 127), 2)

# 이미지 파일 읽기
img = cv2.imread("./faces/newJeans.jpg")
if img is None:
    raise FileNotFoundError("이미지를 불러올 수 없습니다. 경로를 확인하세요.")
    
# 얼굴 검출 및 처리
result_img = facama.detect_and_process_faces(
    img,
    draw_ypr
)
    
# 결과 이미지 저장
output_path = 'result_ypr.jpg'  # 저장할 경로와 파일 이름
cv2.imwrite(output_path, result_img)

#### 5. 얼굴 별로 랜드마크 점찍기
- 랜드마크 점찍기 콜백 함수(callback function) `draw_landmarks` 정의
- 함수 `detect_and_process_faces`에 원본 이미지, `draw_landmarks`를 전달하여 실행

In [ ]:
# 얼굴 랜드마크 표시 함수 정의
def draw_landmarks(input_image: np.ndarray, face: Face) -> None:

    # 각 랜드마크 좌표에 원 그리기
    landmarks = face.landmark_2d_106
    # Draw each landmark point
    for i in range(landmarks.shape[0]):
        point = landmarks[i]
        x, y = int(point[0]), int(point[1])
        cv2.circle(input_image, (x, y), 1, (0, 255, 0), -1)  # Green color, filled circle

# 이미지 파일 읽기
img = cv2.imread("./faces/newJeans.jpg")
if img is None:
    raise FileNotFoundError("이미지를 불러올 수 없습니다. 경로를 확인하세요.")
    
# 얼굴 검출 및 처리
result_img = facama.detect_and_process_faces(
    img,
    draw_landmarks
)
    
# 결과 이미지 저장
output_path = 'result_landmarks.jpg'  # 저장할 경로와 파일 이름
cv2.imwrite(output_path, result_img)

### 얼굴 바꾸기: 특정인 얼굴 찾기, 얼굴 바꾸기
#### 1. Class `Facama`에 메서드 `recognize_and_process_faces` 추가
- 메서드 `recognize_and_process_faces`는 입력 이미지(`input_image`)를 받아 얼굴을 검출하고 기준 이미지(`ref_image`)의 얼굴과의 유사도를 계산하여 얼굴 인식을 수행합니다. 이후 전달받은 콜백 함수를 실행하여 입력 이미지를 가공하여 반환합니다.

In [ ]:
import numpy as np
from typing import Callable

from insightface.app.common import Face
from sklearn.metrics.pairwise import cosine_similarity

def new_method(
    self,
    input_image: np.ndarray,
    ref_image: np.ndarray,
    f: Callable[[np.ndarray, Face, float, bool], None],
    threshold: float = 0.4,
) -> np.ndarray:
    """
    입력 이미지를 받아 얼굴을 검출하고, 각 얼굴에 대해 지정된 함수 f를 적용하며,
    기준 이미지(ref_image)의 얼굴과의 유사도를 계산하여 얼굴 인식을 수행합니다.

    Parameters:
        input_image (np.ndarray): 처리할 이미지.
        ref_image (np.ndarray): 기준 얼굴 이미지.
        f (Callable[[np.ndarray, Face, float, bool], None]): 각 얼굴에 대해 적용할 함수로,
            인자로 input_image, face, similarity, is_match를 받습니다.
        threshold (float): 얼굴 인식 임계값. 유사도가 이 값 이상이면 동일 인물로 판단합니다.

    Returns:
        np.ndarray: 처리된 이미지를 반환합니다.
    """
    ref_embedding = self.get_faces(ref_image)[0].embedding  # 첫 번째 얼굴의 임베딩 사용

    # 입력 이미지에서 얼굴 검출 및 임베딩 추출
    faces = self.get_faces(input_image)

    # 검출된 얼굴 처리
    for face in faces:
        # 대상 얼굴의 임베딩 추출은 이미 face.embedding에 있음

        # 코사인 유사도 계산
        similarity = cosine_similarity([ref_embedding], [face.embedding])[0][0]

        # 유사도가 임계값 이상이면 동일 인물로 판단
        is_match = similarity >= threshold

        # 지정된 함수 호출
        f(input_image, face, similarity, is_match)

    # 처리된 이미지를 반환
    return input_image

# Facama 클래스에 동적으로 메서드 추가
setattr(Facama, "recognize_and_process_faces", new_method)

#### 2. 특정인 찾기
- 유사도 수치를 출력하고 유사도가 높은 경우 얼굴에 녹색 Box를 그리는 콜백 함수(callback function) `process_recognized_face` 정의
- Class `Facama`의 메서드 `recognize_and_process_faces`에 원본 이미지, 찾을 얼굴 이미지,`process_recognized_face`를 전달하여 실행

In [ ]:
# 기준 얼굴 이미지 로드 (뉴진스 멤버 : 하니)
ref_image = cv2.imread("./faces/hanni.jpg")
if ref_image is None:
    raise FileNotFoundError(f"기준 이미지를 불러올 수 없습니다. 경로를 확인하세요: {ref_img_path}")

# 대상 이미지 로드 (뉴진스 5명)
target_image = cv2.imread("./faces/newJeans.jpg")
if target_image is None:
    raise FileNotFoundError(f"대상 이미지를 불러올 수 없습니다. 경로를 확인하세요: {target_img_path}")

# 얼굴 처리 함수 정의
def process_recognized_face(input_image: np.ndarray, face: Face, similarity: float, is_match: bool) -> None:
    # 얼굴 영역 표시
    bbox = face.bbox.astype(int)
    color = (0, 255, 0) if is_match else (0, 0, 255)  # 동일 인물이면 초록색, 아니면 빨간색
    cv2.rectangle(input_image, (bbox[0], bbox[1]), (bbox[2], bbox[3]), color, 2)
    # 유사도 텍스트 표시
    label = f"{similarity:.2f}"
    cv2.putText(input_image, label, (bbox[0], bbox[1]-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

# 얼굴 인식 및 처리
result_img = facama.recognize_and_process_faces(
    target_image,
    ref_image,
    process_recognized_face,
    threshold=0.4  # 임계값은 필요에 따라 조정하세요
)

# 결과 이미지 저장
output_path = 'result_recognition.jpg'  # 저장할 경로와 파일 이름
cv2.imwrite(output_path, result_img)

#### 3. 특정인 얼굴을 다른 얼굴로 바꾸기
- Face Swap 모델을 메모리에 로드하고 해당 AI 기능을 초기화합니다.

In [ ]:
import insightface

# Face Swap 모델을 메모리에 로드하고 AI 기능을 초기화
swapper = insightface.model_zoo.get_model(
            INSWAPPER_PATH, download=True, download_zip=True
        )

- 얼굴을 다른 사람 얼굴로 바꾸는 콜백 함수(callback function) `swap_recognized_face` 정의
- `Facama`의 메서드  `recognize_and_process_faces`에 원본 이미지, 찾을 얼굴 이미지,`swap_recognized_face`를 전달하여 실행

In [ ]:
# 기준 얼굴 이미지 로드 (뉴진스 멤버 : 하니)
ref_image = cv2.imread("./faces/hanni.jpg")
if ref_image is None:
    raise FileNotFoundError(f"기준 이미지를 불러올 수 없습니다. 경로를 확인하세요: {ref_img_path}")

# 대상 이미지 로드 (뉴진스 5명)
target_image = cv2.imread("./faces/newJeans.jpg")
if target_image is None:
    raise FileNotFoundError(f"대상 이미지를 불러올 수 없습니다. 경로를 확인하세요: {target_img_path}")

# 바꿀 얼굴 이미지 로드 (텔런트 : 우현)
swap_image = cv2.imread("./faces/woohyun.jpg")
if swap_image is None:
    raise FileNotFoundError(f"대상 이미지를 불러올 수 없습니다. 경로를 확인하세요: {target_img_path}")

swap_face = facama.get_faces(swap_image)[0]  # 첫 번째 얼굴

# 얼굴 처리 함수 정의
def swap_recognized_face(input_image: np.ndarray, face: Face, similarity: float, is_match: bool) -> None:
    # 얼굴 영역 표시
    bbox = face.bbox.astype(int)
    color = (0, 255, 0) if is_match else (0, 0, 255)  # 동일 인물이면 초록색, 아니면 빨간색

    if is_match:
        input_image[:,:] = swapper.get(input_image, face, swap_face)

    #cv2.rectangle(input_image, (bbox[0], bbox[1]), (bbox[2], bbox[3]), color, 2)
    # 유사도 텍스트 표시
    label = f"{similarity:.2f}"
    cv2.putText(input_image, label, (bbox[0], bbox[1]-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

# 얼굴 인식 및 처리
result_img = facama.recognize_and_process_faces(
    target_image, # (뉴진스 5명)
    ref_image, # (뉴진스 멤버 : 하니)
    swap_recognized_face,
    threshold=0.4  # 임계값은 필요에 따라 조정하세요
)

# 결과 이미지 저장
output_path = 'result_swap.jpg'  # 저장할 경로와 파일 이름
cv2.imwrite(output_path, result_img)